# Import Package

In [2]:
import psycopg2
import pandas as pd

from sqlalchemy import create_engine

# Load Connecting

In [3]:
# Connection details
host_p='horton.db.elephantsql.com'
database_p='gcxukoyd'
user_p='gcxukoyd'
password_p='QifGNuuzNymyq1GzStWeUEDI133MS2h3'
port_p = 5432

# #establishing the connection
conn = psycopg2.connect(database=database_p, user=user_p, password=password_p, host=host_p, port=port_p)
#Creating a cursor object using the cursor() method
conn.autocommit = True 
cursor = conn.cursor()

connection_string = f'postgresql://{user_p}:{password_p}@{host_p}:{port_p}/{database_p}'
engine_postgresql = create_engine(connection_string)

# Soal 1

Soal Filter : Tampilkan data customer dengan kriteria atau filter<br>


*   age harus di bawah rata-rata age seluruh customer, misal rata-rata age adalah 20, maka filter pertama menampilkan data dengan age < 20. **NOTE : cari dulu rata-rata age dengan query/sql**
*   gender hanya M (male)
*   tahun kelahiran harus di bawah 1990-01-01




In [4]:
q='''
SELECT * FROM dim_customer
WHERE birthdate <=  '1990-01-01'
and gender = 'M'
and age <= (SELECT AVG(age) 
    FROM dim_customer)
'''

df = pd.read_sql(q,engine_postgresql)
print(df.shape)
df.head()

(154, 9)


,customer_id,geography_id,customer_name,birthdate,maritalstatus,gender,datefirstpurchase,age,grouping_age
0,11013,543,Ian Jenkins,1988-08-06,M,M,2018-10-15,34,Group 21 - 40
1,11094,33,Cedric Ma,1982-04-01,S,M,2018-10-09,40,Group 21 - 40
2,11095,13,Chad Kumar,1982-09-01,S,M,2016-10-02,40,Group 21 - 40
3,11096,12,Andrés Anand,1982-08-10,M,M,2016-10-12,40,Group 21 - 40
4,11104,23,Edgar Sara,1984-03-11,M,M,2016-11-04,38,Group 21 - 40


# Soal 2

Buat lah Query dengan tampilan data seperti di bawah ini

In [5]:
q='''
SELECT fs.order_detail_id, dp.product_name,dc.customer_name,dg.city,dt.region
FROM fact_sales fs
JOIN dim_customer dc
  ON dc.customer_id = fs.customer_id
JOIN dim_product dp
  ON dp.product_id = fs.product_id
JOIN dim_territory dt
  ON dt.territory_id = fs.territory_id
JOIN dim_geography dg
  ON dg.geography_id = dc.geography_id
'''

df = pd.read_sql(q,engine_postgresql)
print(df.shape)
df.head()

(5953, 5)


,order_detail_id,product_name,customer_name,city,region
0,SO43698-1,"Mountain-100 Silver, 44",Rachael Martinez,Pantin,France
1,SO43704-1,"Mountain-100 Black, 48",Julio Ruiz,East Brisbane,Australia
2,SO43705-1,"Mountain-100 Silver, 38",Curtis Lu,East Brisbane,Australia
3,SO43713-1,"Road-150 Red, 62",Sydney Rogers,Burbank,Southwest
4,SO43714-1,"Road-150 Red, 44",Latasha Alonso,York,United Kingdom


# Soal 3

Tampilkan omset rata-rata perbulan dengan order month descending

In [ ]:
q='''
SELECT MONTH(order_date) AS month, SUM(omset) AS total_omset
FROM fact_sales
GROUP BY month DESC

'''
df = pd.read_sql(q,engine_postgresql)
print(df.shape)
df

(12, 2)


,month,total_omset
0,12.0,5.341223e+06
1,11.0,4.851475e+06
2,10.0,5.012182e+06
3,9.0,4.603846e+06
4,8.0,4.966883e+06
5,7.0,7.129074e+06
6,6.0,5.888909e+06
7,5.0,6.011666e+06
8,4.0,6.601118e+06
9,3.0,9.767824e+06


# Soal 4

Tampilkan data total omset antara laki-laki vs wanita


*   age harus lebih 30 tahun
*   category product adalah Bikes



In [6]:
q='''
SELECT dc.gender, SUM(fs.omset) AS total
FROM fact_sales fs
JOIN dim_customer dc
  ON dc.customer_id = fs.customer_id
JOIN dim_product dp
  ON dp.product_id = fs.product_id
WHERE dc.age > 30 AND dp.category = 'Bikes'
GROUP BY dc.gender
'''

df = pd.read_sql(q,engine_postgresql)
print(df.shape)
df

(2, 2)


,gender,total
0,M,1.211210e+10
1,F,1.376050e+10
